In [426]:
import requests
import pandas as pd
import numpy as np
import json
import time
import os
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

In [407]:
# 1 这里写你自己的headers或者多写几个，避免
headers = {"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"}

''' 1. city: 获取各个地址的天气存储地址的URL '''
def citys():
    # 这里存天气 地址代号，是该地址下载的 
    city_file =open( r"E:\公司数据\2018总\31-天气\_city.json", encoding='utf-8')
    res = city_file.read()
    df = json.loads(res)
    dfs = pd.DataFrame(df)
    dfs2 = dfs[dfs['city_code']!=''].reset_index(drop=True)
    url01 = 'http://t.weather.sojson.com/api/weather/city/'
    dfs2['url'] =  'http://t.weather.sojson.com/api/weather/city/'+dfs2['city_code']
    return dfs2

''' 2. get_one_json: 获取一个URL里的内容 '''
def get_one_json(url):
    git_url = url
    r = requests.get(git_url, auth=('user', 'pass'))
    json = r.json()
    return json
''' 3. get_all_json：获取所有url里的内容，并保存，我是先下载后处理数据  '''
def get_all_json(urls):
    d_json = []
    # 这里是爬取的数据的下载地址
    with open(r'F:\\Learn\\python\\pycodes\\zhengt\\01shade\\get_weater\\start190622.json','a') as fp:
        for i,url in enumerate(urls):
            print('开始抓取第'+str(i)+'个地址数据')
            try:
                data = get_one_json(url)
                fp.write(json.dumps(data,indent=4,ensure_ascii=False))
                d_json.append(data)
                time.sleep(random.randint(3,10))
            except:
                print('抓取过于频繁，有封号风险，目前第'+str(i)+'个地址数据没有抓取到')
    return d_json
''' 4. start：执行函数1,2,3  '''
def start():
    df = citys() 
    path = df['url'].unique()
    data_all = get_all_json(path)
    return data_all

''' 5. save_data：将json数据保存到自己本地中'''
def save_data(data_all):
    js = json.dumps(datas)
    file = open('F:\\Learn\\python\\pycodes\\zhengt\\01shade\\get_weater\\start190622.txt','w')
    file.write(js)
    file.close() 
    
''' 6. get_17_data：数据中是否有城市id以及对应的数据，如果不是这种格式则保存错误的代号'''    
def get_17_data(datas):
    data_a = []
    error = []
    for i, data in enumerate(datas):
        try:
            data_a.append([data['cityInfo']['cityId'],data['data'],data['date']])
        except:
#             print('出错的数据是:',i, data)
            error.append(i)
    return data_a, error

''' 7. get_data：对没有报错的数据进行解析，解析为格式固定的数据'''
'''解析为未来15天，历史1天和当天的天气数据''' 
def get_data(datas):
    datas_fore = pd.DataFrame()
    datas_history = pd.DataFrame()
    datas_today = pd.DataFrame()
    key = ['shidu', 'pm25', 'pm10', 'quality', 'wendu', 'ganmao']
    error =[]
    for data in datas:
        try:
            '''1  未来的数据'''
            forecast01 = data[1]['forecast']
            forecast02 = pd.DataFrame(forecast01)
            forecast02['city_id'] = data[0]
            datas_fore = datas_fore.append(forecast02)
            '''2 历史的数据'''
            history01 = data[1]['yesterday']
            history01['city_id'] = data[0]
            datas_history = datas_history.append([history01])
            '''3 当日数据'''
            today01 = {str(i):data[1][i] for i in key }
            today01['city_id'] = data[0]
            today01['today'] = data[2]
            datas_today = datas_today.append([today01])
        except:
            error.append(data[0])
    return datas_fore, datas_history, datas_today, error

In [ ]:
'''8 如果需要把数据保留到数据库中，我做了如下操作'''

def to_sql(df,to_filename):
    # db = "ChintBI", "household_plant"
    engine = create_engine(str(r"mysql+pymysql://用户名:密码@地址:端口号/数据库?charset=utf8"))
    df.to_sql(to_filename, con=engine, if_exists='append', index=False)    
    
def begin_insert_data(df_fore, df_history, df_today):
    ''' 去重'''
    df_fore.drop_duplicates(inplace=True)
    df_history.drop_duplicates(inplace=True)
    df_today.drop_duplicates(inplace=True)
    '''插入数据库'''
    print('未来天气数据开始插入：=============')
    to_sql(df_fore,"fore")
    print('历史天气数据开始插入：=============')
    to_sql(df_history,"history")
    print('当日天气数据开始插入：=============')
    to_sql(df_today,"today")
    print('数据插入完成：=============')


In [415]:
if __name__=="__main__":
    datas = start()   # 开始下载数据
    save_data(datas)  #  保存数据，
    # 打开刚刚保存的地址里的数据
    file = open('F:\\Learn\\python\\pycodes\\zhengt\\01shade\\get_weater\\start190622.txt','r')
    js = file.read()
    city = citys()
    error01_code = [city['city_code'][i] for i in error01_none]  #  报错的数据
    df_fore, df_history, df_today, error02 =get_data(datas_2)    #  解析数据 
    error = error01_code+error02                                 #  所有爬取错误的城市代码
    begin_insert_data(df_fore, df_history, df_today)             #  数据插入数据库中